In [ ]:
import sys
import os
from env.env1D import env1D
import random #nur zum testen
import neat
import pybullet as p
import time
import math
import numpy as np
import matplotlib.pyplot as plt
from pyquaternion import Quaternion

def getGoalPosition(upperRad, lowerRad, upperTheta, lowerTheta, upperPhi, lowerPhi, num): #Minimum Radius = 0.465, Höhe Gelenk = 0.36, MaxRadius = 0.921
    goalPos = []
    goalSpher = []
    for i in range(num):
        rad = random.uniform(lowerRad, upperRad)
        theta = random.uniform(lowerTheta, upperTheta)
        #phi = random.uniform(lowerPhi, upperPhi)
        phi = 90 # x-Koordinate = 0
        zOffset = 0.36 #Höhe des Gelenkes
        x = math.sin(math.radians(theta)) * math.cos(math.radians(phi)) * rad
        y = math.sin(math.radians(theta)) * math.sin(math.radians(phi)) * rad
        z = math.cos(math.radians(theta)) * rad + zOffset
        goal = [x,y,z]
        goalS = [rad, math.radians(theta), math.radians(phi)]
        goalPos.append(goal)
        goalSpher.append(goalS)
    print(goalPos)
    print(goalSpher)
    return goalPos, goalSpher

def getGoalRad():
    goalPos=[]
    goalSpher = []
    maxRad = 0.921
    minRad = 0.465
    radDiff = maxRad - minRad
    theta = 45
    phi = 90
    zOffset = 0.36    
    num = 90 # Anzahl der Punkte
    for i in range(num):
        rad = maxRad - (i * radDiff/num)
        x = math.sin(math.radians(theta)) * math.cos(math.radians(phi)) * rad
        y = math.sin(math.radians(theta)) * math.sin(math.radians(phi)) * rad
        z = math.cos(math.radians(theta)) * rad + zOffset
        goal = [x,y,z]
        goalS = [rad, math.radians(theta), math.radians(phi)]
        goalPos.append(goal)
        goalSpher.append(goalS)
    return goalPos, goalSpher

def evalFitness(isState, basePos, goalPos):
    x = isState[0] - basePos[0]
    y = isState[1] - basePos[1]
    z = isState[2] - basePos[2]
    goalX = goalPos[0]
    goalY = goalPos[1]
    goalZ = goalPos[2]
    #print('aktueller Winkel: ', isAngle )
    distance = math.sqrt(((goalZ-z)**2)+((goalY-y)**2)+((goalX-x)**2))
    print(distance)
    maxDistance = 0.921
    score = ((maxDistance - distance)/maxDistance) * 100
    return score
#Roboter Steuern über die Tasten N,M für das unterste Gelenk; J und K mittleres Gelenk; I und O oberes Gelenk
#C berechnet die Fitness für den aktuellen Punkt und gibt die Differenz zum gesuchten Punkt für alle Achsen aus
#Die Simulation muss das geöffnete Fenster sein, damit die Tastenaktionen ausgeführt werden
#
#In env1D die JointPosition auf math.pi/2 setzen, damit erhält man für Z die höhe des Gelenkes und mit y den maximalen Radius
goal, goalSpher = getGoalPosition(0.921, 0.467, 0, -90, 169, -169, 1)
print(goal[0])
p.connect(p.GUI)
p.setGravity(0,0,0)
bot = env1D(startPos = [0,0,0])
print(bot.upperLimits)
print(bot.lowerLimits)
spher = bot.getBotSpherical()
print(goalSpher)
run = True
joint = 0
while(run):

    events = p.getKeyboardEvents()
    if len(events) != 0:
        key = list(events.keys())[0]
        #print(key)
        if key == 65295:  #left
            bot.moveDir[0] = 1
            bot.moveStep()
        elif key == 65296: #right
            bot.moveDir[0] = -1
            bot.moveStep()
        elif key == 65297: #up
            bot.moveDir[1] = -1
            bot.moveStep()
        elif key == 65298: #down
            bot.moveDir[1] = 1
            bot.moveStep()
        elif key == 108: #l
            bot.moveDir[2] = -1
            bot.moveStep()
        elif key == 107: #K
            bot.moveDir[2] = 1
            bot.moveStep()
        elif key == 110: #N
            bot.moveDir[3] = -1
            bot.moveStep()
        elif key == 109: #M
            bot.moveDir[3] = 1
            bot.moveStep()
        elif key == 104: #H
            bot.moveDir[4] = -1
            bot.moveStep()
        elif key == 106: #J
            bot.moveDir[4] = 1
            bot.moveStep()
        elif key == 117: #U
            bot.moveDir[5] = -1
            bot.moveStep()
        elif key == 105: #I
            bot.moveDir[5] = 1
            bot.moveStep()
        elif key == 111: #O
            bot.moveDir[6] = -1
            bot.moveStep()
        elif key == 112: #P
            bot.moveDir[6] = 1
            bot.moveStep()
        elif key == 99:
            spher = bot.getBotSpherical()
            Q1 = p.getLinkState(bot.botId, 6)[1]
            q1 = Quaternion(Q1[3], Q1[0], Q1[1], Q1[2])
            E1 = p.getEulerFromQuaternion(Q1)
            print('Quaternion:', q1)
            print('Euler', E1)
            #fitness = evalFitness(p.getLinkState(bot.botId,bot.numJoints-1)[0], bot.startPosition, goal[0])
            #print (fitness)
            print('x: ', p.getLinkState(bot.botId,bot.numJoints-1)[0][0],
                 'y: ', p.getLinkState(bot.botId,bot.numJoints-1)[0][1],
                 'z: ', p.getLinkState(bot.botId,bot.numJoints-1)[0][2])
            print('Spherical ', bot.getBotSpherical())
            #print('Joint1 Pos:', p.getJointState(bot.botId, 1))
            #print('zPos', p.getLinkState(bot.botId,bot.numJoints-1)[0][2])
            #print('diff', p.getLinkState(bot.botId,bot.numJoints-1)[0][2]-p.getLinkState(bot.botId,0)[0][2])
        elif key == 120:  
            headPosSpher = bot.getBotSpherical()
            headPos = bot.getBotHeadPos()
            print(headPosSpher)
            print(headPos)
            #Q2 = p.getLinkState(bot.botId, 6)[1]
            #q2 = Quaternion(Q2[3], Q2[0], Q2[1], Q2[2])
            #E2 = p.getEulerFromQuaternion(Q2)
            #print('Quaternion', Q2)
            #print('Euler', E2)
            #headEuler = p.getEulerFromQuaternion(headOr)
            #print('Euler', headEuler)
            #dist = Quaternion.absolute_distance(q1, q2)
            #print('Distance: ', dist)
            #print('diff Rad: ', headPosSpher[0]-goalSpher[0][0])
            #print('goalTheata: ', goalSpher[0][1])
            #print('HeadTheata:', headPosSpher[1])
            #print('diff Theta: ', headPosSpher[1]-goalSpher[0][1])
        for i in range(6):
            bot.moveDir[i] = 0
    p.stepSimulation()  
    time.sleep(1/240)

p.disconnect()


[[-2.1909567976323834e-17, -0.3578103987464488, 0.806162995142644]]
[[0.5719175637150408, -0.6759424558974635, 1.5707963267948966]]
[-2.1909567976323834e-17, -0.3578103987464488, 0.806162995142644]
[2.96705972839, 2.09439510239, 2.96705972839, 2.09439510239, 2.96705972839, 2.09439510239, 3.05432619099]
[-2.96705972839, -2.09439510239, -2.96705972839, -2.09439510239, -2.96705972839, -2.09439510239, -3.05432619099]
[[0.5719175637150408, -0.6759424558974635, 1.5707963267948966]]
Quaternion: 0.500 +0.000i -0.866j +0.000k
Euler (-3.141592523629104, -1.0471995770802014, 3.141592648642762)
x:  -0.43388012393377556 y:  -6.296369584301567e-08 z:  0.5295024169802689
Spherical  [0.4658143743026243, 1.1983618996017014, -3.14159250847207]
Quaternion: 0.500 +0.000i -0.866j +0.000k
Euler (-3.1415925035587446, -1.047199201969411, 3.1415926028679446)
x:  -0.4338799747012449 y:  -5.3157908334704185e-08 z:  0.5295021586414922
Spherical  [0.46581414129551857, 1.198362299599934, -3.1415925310722503]
Quater

import Training7RRel
Training7RRel.start()
